In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
)

In [12]:
# Basic text generation
response = llm.invoke("What is ML?")
print(response.content)

ML stands for **Machine Learning**.

In essence, Machine Learning is a field of computer science that gives computers the ability to learn from data without being explicitly programmed.  Instead of writing code that tells a computer exactly what to do, you feed it data, and the machine learning algorithm learns patterns and relationships within that data.  It then uses these learned patterns to make predictions or decisions on new, unseen data.

Here's a breakdown of key aspects:

*   **Learning from Data:** The core of ML is the ability to improve performance on a specific task by learning from experience, which comes in the form of data. This data can be anything from numbers and text to images and videos.

*   **Algorithms:** ML uses various algorithms to learn from data. Some common examples include:
    *   **Linear Regression:**  Predicting a continuous output based on linear relationships.
    *   **Logistic Regression:** Predicting a categorical output (e.g., yes/no, spam/not s